In [1]:
import os
import pandas as pd

data_dir = os.path.join('.', 'data', 'blender-off-multiview-tool')
DESIRED_SIZE = 224

In [2]:
df = pd.read_hdf(os.path.join('labels', 'REORGANIZE-PART-SHREC14', 'sk_orig.hdf5'))
paths = df.index.values
sk_paths = []
for p in paths:
    sk_paths.append(os.path.join(data_dir, p))
print('total number of sketches: ', len(sk_paths))

cad_im_path = os.path.join(data_dir, 'dataset_3d_model_view_224x224')
cad_class = os.listdir(cad_im_path)
cad_paths = []
for c in cad_class:
    class_path = os.path.join(cad_im_path, c)
    class_item = os.listdir(class_path)
    for item in class_item:
        item_path = os.path.join(class_path, item)
        views = os.listdir(item_path)
        for v in views:
            cad_paths.append(os.path.join(item_path, v))
print('total number of 3D models\' views: ', len(cad_paths))

total number of sketches:  3840
total number of 3D models' views:  86856


In [3]:
import numpy as np
from PIL import Image, ImageOps

def transform_im(im):
    '''
    Scale the picture to prevent distortion
    '''
    if im.mode == 'RGBA':
        im = im.convert('RGB')
    # [width, height]
    old_size = np.asarray(im.size)
    ratio = float(DESIRED_SIZE) / max(old_size)
    new_size = map(int, old_size * ratio)
    # Image.NEAREST ：最近邻居
    # Image.BILINEAR：双线性差值
    # Image.BICUBIC ：双三次插值法
    # Image.ANTIALIAS：面积插值法
    im = im.resize(new_size, Image.ANTIALIAS)
    return im


def process_im(path, mode):
    im = Image.open(path)
    im = transform_im(im)
    # saving
    im_dir = os.path.dirname(path)
    fname = os.path.basename(path)
    split = im_dir.split(os.path.sep)[-1]
    clsname = im_dir.split(os.path.sep)[-2]
    
    save_dir = ''
    if mode == 'sketch':
        save_dir = os.path.join(*im_dir.split(os.path.sep)[:-4])
        save_dir = os.path.join(save_dir, 'SHREC14LSSTB_SKETCHES_RESIZED', clsname, split)
    else:
        save_dir = os.path.join(data_dir, 'SHREC14', 'SHREC14LSSTB_TARGET_MODELS_RESIZED', clsname, split)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # The quality of the saved image, with values ranging from 1 (worst) to 95 (best)
    im.save(os.path.join(save_dir, fname), quality=95)

In [4]:
print('----- start sketches resize -----')
for i in range(len(sk_paths)):
    process_im(sk_paths[i], 'sketch')
    if i and i % 500 == 0:
        print('done:', i)
print('total:', len(sk_paths))
print('----- all be done -----')

print('----- start 3D models\' views resize -----')
for i in range(len(cad_paths)):
    process_im(cad_paths[i], '3d')
    if i and i % 5000 == 0:
        print('done:', i)
print('total:', len(cad_paths))
print('----- all be done -----')

----- start sketches resize -----
done: 500
done: 1000
done: 1500
done: 2000
done: 2500
done: 3000
done: 3500
total: 3840
----- all be done -----
----- start 3D models' views resize -----
done: 5000
done: 10000
done: 15000
done: 20000
done: 25000
done: 30000
done: 35000
done: 40000
done: 45000
done: 50000
done: 55000
done: 60000
done: 65000
done: 70000
done: 75000
done: 80000
done: 85000
total: 86856
----- all be done -----
